In [1]:
import numpy as np
import xsimlab as xs
import xarray as xr

import fastscape
from fastscape.models import basic_model
from fastscape.processes import TwoBlocksUplift
from ipyfastscape import TopoViz3d

/Users/jiao/miniconda/lib/python3.7/site-packages/xsimlab/variable.py:211: FutureWarning: Setting variable group using `group` is depreciated; use `groups`.
  "groups": _as_group_tuple(groups, group),


In [2]:
uplift_rate = .2e-3 # meter per year
uplift_duration = 6e6 # year
model_length_x = 50e3 # meter
model_length_y = 50e3 # meter

model = basic_model.update_processes({'uplift': TwoBlocksUplift})

In [3]:
in_ds = xs.create_setup(
    model=model,
    clocks={
        'clock': np.arange(0, uplift_duration + 10e3, 10e3),
        'time': np.arange(0, uplift_duration + 10e3, 10e3),
    },
    master_clock='time',
    input_vars={
        'grid__shape': [int(model_length_y/500), int(model_length_x/500)],
        'grid__length': [model_length_y, model_length_x],
        'boundary__status': ['fixed_value', 'looped', 'looped', 'looped'],
        'uplift': {
            'x_position': model_length_x*.25,
            'rate_left': 0.,
            'rate_right': uplift_rate
        },
        'spl': {
            'k_coef': 2e-6,
            'area_exp': 0.5,
            'slope_exp': 1.
        },
        'diffusion__diffusivity': 1e-1
    },
    output_vars={'topography__elevation': 'time',
                'drainage__area': 'time',
        None: ['boundary__border',
               'grid__x',
               'grid__y',
               'grid__spacing'],
    }
)

/Users/jiao/miniconda/lib/python3.7/site-packages/xsimlab/xr_accessor.py:582: FutureWarning: Setting clock dimensions or `None` as keys for `output_vars` is depreciated; use variable names instead (and clock dimensions or `None` as values, see docs).
  ds.xsimlab._set_output_vars(model, output_vars)


In [4]:
with xs.monitoring.ProgressBar():
    output = (in_ds.xsimlab.run(model=model)
               .set_index(x='grid__x', y='grid__y',
                          border='boundary__border'))

In [5]:
v1 = TopoViz3d(output, canvas_height=600, time_dim='time')
v1.show()

Output(layout=Layout(height='640px'))